In [62]:
import sys, os
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine

constr = 'mysql+pymysql://{user}:{pwd}@{host}'.format(user=os.environ['MYSQL_USERNAME'],
                                                      pwd=os.environ['MYSQL_PASSWORD'],
                                                      host=os.environ['MYSQL_HOST'])
con = create_engine(constr)

In [27]:
# df = pd.read_csv('first_edit_all_uk_pages.csv', parse_dates=True)

In [48]:
q = """
select r.rev_page as page_id, min(r.rev_timestamp) as first_edit
from ukwiki_p.revision as r
join ukwiki_p.page as p
on p.page_id = r.rev_page
where p.page_namespace=0
group by r.rev_page;
"""

In [49]:
df = pd.read_sql(q, con)

In [53]:
df['first_edit'] = df.first_edit.str.decode("utf-8")

In [54]:
df['first_edit'] = pd.to_datetime(df['first_edit'], format="%Y%m%d%H%M%S")

In [55]:
df.head()

,page_id,first_edit
0,2,2003-12-23 09:53:53
1,3,2003-12-27 00:45:00
2,6,2003-10-15 10:51:10
3,9,2004-01-25 14:56:26
4,13,2004-01-26 11:04:32


In [56]:
# end date = current date
df['end_date'] = datetime.now()

In [57]:
df.first_edit[0].date()

datetime.date(2003, 12, 23)

In [58]:
df['days_old'] = (df['end_date'] - df['first_edit']).dt.days

In [102]:
df.head()

,page_id,first_edit,end_date,days_old
0,2,2003-12-23 09:53:53,2018-07-18 12:56:05.606391,5321
1,3,2003-12-27 00:45:00,2018-07-18 12:56:05.606391,5317
2,6,2003-10-15 10:51:10,2018-07-18 12:56:05.606391,5390
3,9,2004-01-25 14:56:26,2018-07-18 12:56:05.606391,5287
4,13,2004-01-26 11:04:32,2018-07-18 12:56:05.606391,5287


In [61]:
df.to_csv('first_edit_all_uk_pages.csv', index=False)

In [116]:
q = """
select r.rev_page as page_id, count(distinct rev_user_text) as contributions
from ukwiki_p.revision as r
join ukwiki_p.page as p
on p.page_id = r.rev_page
where p.page_namespace=0
group by r.rev_page;
"""

In [117]:
contributions = pd.read_sql(q, con)

In [118]:
contributions.head()

,page_id,contributions
0,2,4
1,3,72
2,6,5
3,9,4
4,13,156


In [119]:
contributions.to_csv('contributions_by_username_or_ip_all_uk_pages.csv', index=False)